In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
#큰 과정은 머신러닝을 돌리는건데, 피쳐를 추가하려다보니 딥러닝을 이용했다.

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import warnings
warnings.filterwarnings("ignore")
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

#Neural Network 모델링(딥러닝 모델) => 인간에게 한계가 있기 때문에
train = pd.read_csv('../input/train.csv', parse_dates=['Dates'])
test = pd.read_csv('../input/test.csv', parse_dates=['Dates'], index_col='Id')

#각각에 들어가 있는 단어의 갯수가 다르다. 딥러닝 모델에 넣을려면 각각 토크나이징 해야함
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(list(train["Address"]) + list(test["Address"]))

haha = tokenizer.texts_to_sequences(train["Address"]) # 4개짜리, 5개짜리도 있다.. 칼럼의 수를 맞쳐줘야 딥러닝 모델에 input으로 넣을 수 있다.
haha2 = tokenizer.texts_to_sequences(test["Address"])

#단어의 갯수를 동일하게 맞쳐주기 위해(딥러닝 모델에 input에 넣기 위해), 앞에 또는 뒤에 0을 padding 시켜줄 수 있음.
from keras.preprocessing.sequence import pad_sequences
padded = pad_sequences(haha, maxlen=7)
padded2 = pad_sequences(haha2, maxlen=7)

#Sequential 층층이 쌓을때
#1. 모델을 불러온다
from keras import Sequential
from keras.layers import Flatten, Embedding, Dense 
#2. 모델정의
model = Sequential()
#3. 3개의 층이 들어감
# 1. 임베딩층 
model.add(Embedding(2201, 1, input_length=7)) #2201의 의미: 유니크한 갯수 : len(tokenizer.word_index)  #input_length = 7: 단어 들어오는 갯수

# 2. 차원축소 층
model.add(Flatten()) #1차원층으로 바꿔준다.
# 3. 출력층(Dense Layer)
model.add(Dense(39, activation='softmax')) # 39의 의미: 범주카데고리 클래쓰의 개수, activation='softmax' 확률 값으로 알려줌(맨 마지막 activation 함수는 무조건 softmax 다중 분류 문제이기때문에)

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['acc']) #optimizer = 최적화 함수 (대부분 adam 쓰면됨 - 학습속도 빠

#학습
from sklearn.preprocessing import LabelEncoder
le2 = LabelEncoder()
y= le2.fit_transform(train['Category'])
model.fit(padded, y, epochs = 10 , batch_size = 2048)

preds = model.predict(padded)
preds2 = model.predict(padded2)

# 차원 축소
from sklearn.decomposition import PCA

pca1 = PCA(n_components=2)
X_low = pca1.fit_transform(preds)
X_low2 = pca1.transform(preds2)

train['Date'] = train['Dates'].dt.date
#1. feature engineering : n_days 칼럼
train['n_days'] = (train['Date'] - train['Date'].min()).apply(lambda x: x.days) #최초 시점으로 부터 얼마나 지났는 지
train['Day'] = train['Dates'].dt.day
train['DayOfWeek'] = train['Dates'].dt.weekday
train['Month'] = train['Dates'].dt.month
train['Year'] = train['Dates'].dt.year
train['Hour'] = train['Dates'].dt.hour
train['Minute'] = train['Dates'].dt.minute
train['Block'] = train['Address'].str.contains('block', case=False)
train['ST'] = train['Address'].str.contains('ST', case=False)
train["X_Y"] = train["X"] - train["Y"]
train["XY"] = train["X"] + train["Y"]

test['Date'] = test['Dates'].dt.date
test['n_days'] = (test['Date'] - test['Date'].min()).apply(lambda x: x.days)
test['Day'] = test['Dates'].dt.day
test['DayOfWeek'] = test['Dates'].dt.weekday
test['Month'] = test['Dates'].dt.month
test['Year'] = test['Dates'].dt.year
test['Hour'] = test['Dates'].dt.hour
test['Minute'] = test['Dates'].dt.minute
#2 feature engineering: 주소 관련 칼럼 ex) 서울시 용산구 원효로 10 - 서울시 용산구 원효로 7 거의 비슷하게 인지(label encoding시)
test['Block'] = test['Address'].str.contains('block', case=False) #case=False => 대문자 소문자 전부 추출
test['ST'] = test['Address'].str.contains('ST', case=False)
# tree model의 단점: 각각 feature간의 같이 만들어내는 상호작용을 잘 해석 못시킴 ex) 키와 몸무게로 만들어지는 BMI 지수 같은거 잘 해석 못함
#3 feature engineering: 위도,경도    위도- 경도, 위도+경도 => feature을 추가해줌으로써 훨씬 더 데이터를 세분화해서 디테일할수 있게 학습가능 , 새로운 백터의 축 개념이 생기면서, 거리와 공간의 개념이 생김
test["X_Y"] = test["X"] - test["Y"]
test["XY"] = test["X"] + test["Y"]

# Label Encoding 
from sklearn.preprocessing import LabelEncoder
le1 = LabelEncoder()
train['PdDistrict'] = le1.fit_transform(train['PdDistrict'])
test['PdDistrict'] = le1.transform(test['PdDistrict'])

le2 = LabelEncoder()
y= le2.fit_transform(train['Category'])

train = pd.concat([train, pd.DataFrame(X_low)], 1)
test = pd.concat([test, pd.DataFrame(X_low2)], 1)

le3 = LabelEncoder()
le3.fit(list(train['Address']) + list(test['Address']))
train['Address'] = le3.transform(train['Address'])
test['Address'] = le3.transform(test['Address'])

train.drop(['Dates','Date','Descript','Resolution', 'Category'], 1, inplace=True)
test.drop(['Dates','Date',], 1, inplace=True)

train.to_csv("train.csv", index=False)
test.to_csv("test.csv", index=False)


# 1. 모델을 불러온다
from lightgbm import LGBMClassifier

# 2. 모델을 선언한다. 
# 하이퍼파라미터 하는이유: 데이터셋의 특성이 항상 다르기 때문에, 
#LGBM 모델 선정 이유 1. 문자열 데이터를 잘 처리함(label encoding+tree모델) 2. 데이터가 대용량 사용 가능(데이터가 크면 클수록 속도가 더 빨라지는 특징) 3. 카데고리형 문제가 많아서 트리모델을 써야한다, 다른모델을 거리 기반으로 보통 학습한다., 트리모델은 거리가 있어도, 데이터를 분류할때 정보이득이 높고, 엔트로피가 낮아지는 방향으로 알아서 세분화 .
#다른모델들은 onehot-encoding을 쓴다 (unique한 클래스를 만듬) - 차원의 크기가 커지는 단점이 있다. (onehot-encoding은 왠만하면 안쓰는게 좋다)

#하이파라미터 
#colsample_bytree :(기본 0.8 => 칼럼이 너무 많은 경우 값을 조금 줄여도 됨:0.5~0.8  ) 나무당 칼럼을 얼마나 학습할 것인가(전체데이터셋에서 몇퍼센트씩 랜덤방식으로 학습) => 1. overfiting 문제를 간접적으로 막을 수 있음  2. 모델 학습 속도가 빨라짐 (당연히 각 나무에 들어가는 데이터의 칼럼의수가 적어지므로) 3. 다양성 효과로 앙삼블 효과가 있음(tree모델은 굉장히 탐욕적, 학습잘하는 칼럼을 찾으면 그곳으로 가중치주면서 집중학습시작 (overfiting 잘되는 경향), 데이터 전체로만 학습하면 모든 나무에 다양성이 없어짐 
#subsample : (기본 0.7 => 데이터가 많은경우: 0.6~0.7)나무당 열을 얼마나 학습할것인가 => 1. overfiting 문제를 간접적으로 막을 수 있음  2. 모델 학습 속도가 빨라짐 (당연히 각 나무에 들어가는 데이터의 칼럼의수가 적어지므로) 3. 다양성 효과로 앙삼블 효과
#num_leaves : (기본 31 => 데이터가 복잡할때(카데고리 갯수 많거나, 데이터양이 많을때) 숫자를 높여야함) LGBM에서 제일 중요한 파라미터로써, 만들어질 수 있는 노드의 갯수(깊어지는 방향을 튜닝가능)  <=> 다른 트리 모델은 max_depth 하이파라미터로 씀 (2^n-1)
#num_leaves 찾는 방법: 데이터 학습시키면서 올려보고 내려보고
#n_estimator: (기본 100개)많이 입력하면 overfit, 적게 입력하면 overfit 
#learning late: 얼마나 빠르게 수렴할지
hyper = {'colsample_bytree': 0.625,
 'is_unbalance': False,
 'learning_rate': 0.025,
 'min_child_samples': 105,
 'num_class': 39,
 'num_leaves': 233,
 'objective': 'multiclass',
 'reg_alpha': 0.4000134592012641,
 'reg_lambda': 0.5082596745249518,
 'subsample': 0.9338693244190213,
 'subsample_for_bin': 140000,
 'n_estimators': 600}
model = LGBMClassifier(**hyper)
model.fit(train, y, categorical_feature=["PdDistrict", "DayOfWeek"])
preds = model.predict_proba(test)
submission = pd.DataFrame(preds, columns=le2.inverse_transform(np.linspace(0, 38, 39, dtype='int16')), index=test.index)
submission.to_csv('LGBM_final.csv', index_label='Id')